In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import AgglomerativeClustering
from sentence_transformers import SentenceTransformer, util

c:\Users\smhue\VisualStudio\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

reviews = pd.read_csv('reviews.csv')


In [3]:
#This is the global variable resturant 
queryResturant = "Haci'nin Yeri - Yigit Lokantasi"

In [4]:
def findAverageRating(restaurant):
    
    averageRating = reviews[reviews['business_name'] == restaurant]['rating'].mean()

    
    return averageRating


In [5]:
findAverageRating(queryResturant)

averageRating = findAverageRating(queryResturant)
averageRating = round(averageRating, 2)

In [6]:
def getReviews(queryResturant):
    reviews = pd.read_csv('reviews.csv')
    reviews = reviews.loc[reviews['business_name'] == queryResturant]
    reviews = reviews['text']
    reviews = reviews.values
    return reviews


In [7]:
reviews = getReviews(queryResturant)


In [8]:
np.random.seed(2022)

from sentence_transformers import SentenceTransformer

model = SentenceTransformer('multi-qa-mpnet-base-dot-v1')

embeddings = model.encode(reviews, convert_to_tensor=True)

In [9]:
top_k = 3
used_indices = set()
usable_corpus = reviews.tolist()
original_cluster_labels = reviews.tolist()
clusters = []
cls_id = 0


while(len(used_indices) < len(usable_corpus)):
    cls_id += 1
    query = usable_corpus[0]
    usable_corpus = [label for label in reviews if label not in used_indices]
    if query in usable_corpus: 
        usable_corpus.remove(query)
    

    #semenatic search
    query_embedding = model.encode(query, convert_to_tensor=True)
    embeddings = model.encode(usable_corpus, convert_to_tensor=True)
    cos_scores = util.pytorch_cos_sim(query_embedding, embeddings)[0]
    
    for score in cos_scores:
        if score > 0.5: 
          top_results = np.argpartition(-cos_scores, range(top_k))[0:top_k]
          top_matches = [usable_corpus[idx] for idx in top_results[0:top_k]]  
        else: 
            continue
    
  

    for idx in top_matches:
        used_indices.add(idx)

    top_matches.append(query)
    used_indices.add(query)

    clusters.append(top_matches)


In [10]:
cluster_labels = []

for cluster in clusters:
    cluster_embeddings = model.encode(cluster, convert_to_tensor=True)
    cos_scores = util.pytorch_cos_sim(cluster_embeddings, cluster_embeddings)
    cos_scores = cos_scores.cpu().detach().numpy()
    cos_scores = np.max(cos_scores, axis=0)
    top_cluster = np.argmax(cos_scores)
    cluster_labels.append(cluster[top_cluster])



In [11]:
#The output 
print("I recommend that you try out this restaurant: ", queryResturant)
print("The average rating of the restaurant is: ", averageRating)

print("From the reviews, the most common experiences are: ")
for label in cluster_labels:
    print("- ", label)

"The overall experience of this restaurant is normally good."
print("This will more than likely be your experience when you go to", queryResturant, ".")
if averageRating >= 4.0:
    print("The overall experience of this restaurant is normally good.")
elif averageRating >= 3.0:
    print("The overall experience of this restaurant is normally average.")
elif averageRating >= 2.0:
    print("The overall experience of this restaurant is normally bad.")
else:
    print("The overall experience of this restaurant is normally awful.")

I recommend that you try out this restaurant:  Haci'nin Yeri - Yigit Lokantasi
The average rating of the restaurant is:  4.45
From the reviews, the most common experiences are: 
-  We went to Marmaris with my wife for a holiday. We chose this restaurant as a place for dinner based on the reviews and because we wanted juicy food. When we first went there was a serious queue. You proceed by taking the food you want in the form of an open buffet. Both vegetable dishes and meat dishes were plentiful. There was also dessert for those who wanted it. After you get what you want you pay at the cashier. They don't go through cards they work in cash. There was a lot of food variety. And the food prices were unbelievably cheap. We paid only 84 TL for all the meals here. It included buttermilk and bread. But unfortunately I can't say it's too clean as a place..
-  What you see is 125 TL in total. It's a pretty convenient place. We can say that it is an artisan restaurant; you should not expect a r